In [31]:
%load_ext autoreload
%autoreload 2

In [4]:
# !pip install --upgrade torchvision
# !pip install --upgrade torch

In [1]:
!mkdir /Users/aziai/Downloads/vtest

In [2]:
!cp /Users/aziai/Downloads/movies_casino_royale_2006.mp4 /Users/aziai/Downloads/vtest/

In [3]:
from data import make_dataset

In [6]:
!ls -lht '/Users/aziai/Downloads/movies_casino_royale_2006_kissing_1.mp4'

-rw-r--r--@ 1 aziai  staff   1.0M May 29 18:53 /Users/aziai/Downloads/movies_casino_royale_2006_kissing_1.mp4


In [9]:
q = make_dataset('/Users/aziai/Downloads/vtest/', 10)
q

[]

In [13]:
import torch
from torch.autograd import Variable

from data import Video
from spatial_transforms import (Compose, Normalize, Scale, CenterCrop, ToTensor)
from temporal_transforms import LoopPadding

def classify_video(video_dir, video_name, class_names, model, opt):
    assert opt.mode in ['score', 'feature']

    spatial_transform = Compose([Scale(opt.sample_size),
                                 CenterCrop(opt.sample_size),
                                 ToTensor(),
                                 Normalize(opt.mean, [1, 1, 1])])
    temporal_transform = LoopPadding(opt.sample_duration)
    data = Video(video_dir, spatial_transform=spatial_transform,
                 temporal_transform=temporal_transform,
                 sample_duration=opt.sample_duration)
    data_loader = torch.utils.data.DataLoader(data, batch_size=opt.batch_size,
                                              shuffle=False, num_workers=opt.n_threads, pin_memory=True)

    video_outputs = []
    video_segments = []
    for i, (inputs, segments) in enumerate(data_loader):
        inputs = Variable(inputs, volatile=True)
        outputs = model(inputs)

        video_outputs.append(outputs.cpu().data)
        video_segments.append(segments)

    video_outputs = torch.cat(video_outputs)
    video_segments = torch.cat(video_segments)
    results = {
        'video': video_name,
        'clips': []
    }

    _, max_indices = video_outputs.max(dim=1)
    for i in range(video_outputs.size(0)):
        clip_results = {
            'segment': video_segments[i].tolist(),
        }

        if opt.mode == 'score':
            clip_results['label'] = class_names[max_indices[i]]
            clip_results['scores'] = video_outputs[i].tolist()
        elif opt.mode == 'feature':
            clip_results['features'] = video_outputs[i].tolist()

        results['clips'].append(clip_results)

    return results

In [77]:
class Opt:
    def __init__(self):
        self.mode = 'score'
        self.batch_size = 128
        self.sample_duration = 21
        self.sample_size = 112
        self.mean = [114.7748, 107.7354, 99.4750]

In [78]:
opt = Opt()

In [19]:
cls = """abseiling
air drumming
answering questions
applauding
applying cream
archery
arm wrestling
arranging flowers
assembling computer
auctioning
baby waking up
baking cookies
balloon blowing
bandaging
barbequing
bartending
beatboxing
bee keeping
belly dancing
bench pressing
bending back
bending metal
biking through snow
blasting sand
blowing glass
blowing leaves
blowing nose
blowing out candles
bobsledding
bookbinding
bouncing on trampoline
bowling
braiding hair
breading or breadcrumbing
breakdancing
brush painting
brushing hair
brushing teeth
building cabinet
building shed
bungee jumping
busking
canoeing or kayaking
capoeira
carrying baby
cartwheeling
carving pumpkin
catching fish
catching or throwing baseball
catching or throwing frisbee
catching or throwing softball
celebrating
changing oil
changing wheel
checking tires
cheerleading
chopping wood
clapping
clay pottery making
clean and jerk
cleaning floor
cleaning gutters
cleaning pool
cleaning shoes
cleaning toilet
cleaning windows
climbing a rope
climbing ladder
climbing tree
contact juggling
cooking chicken
cooking egg
cooking on campfire
cooking sausages
counting money
country line dancing
cracking neck
crawling baby
crossing river
crying
curling hair
cutting nails
cutting pineapple
cutting watermelon
dancing ballet
dancing charleston
dancing gangnam style
dancing macarena
deadlifting
decorating the christmas tree
digging
dining
disc golfing
diving cliff
dodgeball
doing aerobics
doing laundry
doing nails
drawing
dribbling basketball
drinking
drinking beer
drinking shots
driving car
driving tractor
drop kicking
drumming fingers
dunking basketball
dying hair
eating burger
eating cake
eating carrots
eating chips
eating doughnuts
eating hotdog
eating ice cream
eating spaghetti
eating watermelon
egg hunting
exercising arm
exercising with an exercise ball
extinguishing fire
faceplanting
feeding birds
feeding fish
feeding goats
filling eyebrows
finger snapping
fixing hair
flipping pancake
flying kite
folding clothes
folding napkins
folding paper
front raises
frying vegetables
garbage collecting
gargling
getting a haircut
getting a tattoo
giving or receiving award
golf chipping
golf driving
golf putting
grinding meat
grooming dog
grooming horse
gymnastics tumbling
hammer throw
headbanging
headbutting
high jump
high kick
hitting baseball
hockey stop
holding snake
hopscotch
hoverboarding
hugging
hula hooping
hurdling
hurling (sport)
ice climbing
ice fishing
ice skating
ironing
javelin throw
jetskiing
jogging
juggling balls
juggling fire
juggling soccer ball
jumping into pool
jumpstyle dancing
kicking field goal
kicking soccer ball
kissing
kitesurfing
knitting
krumping
laughing
laying bricks
long jump
lunge
making a cake
making a sandwich
making bed
making jewelry
making pizza
making snowman
making sushi
making tea
marching
massaging back
massaging feet
massaging legs
massaging person's head
milking cow
mopping floor
motorcycling
moving furniture
mowing lawn
news anchoring
opening bottle
opening present
paragliding
parasailing
parkour
passing American football (in game)
passing American football (not in game)
peeling apples
peeling potatoes
petting animal (not cat)
petting cat
picking fruit
planting trees
plastering
playing accordion
playing badminton
playing bagpipes
playing basketball
playing bass guitar
playing cards
playing cello
playing chess
playing clarinet
playing controller
playing cricket
playing cymbals
playing didgeridoo
playing drums
playing flute
playing guitar
playing harmonica
playing harp
playing ice hockey
playing keyboard
playing kickball
playing monopoly
playing organ
playing paintball
playing piano
playing poker
playing recorder
playing saxophone
playing squash or racquetball
playing tennis
playing trombone
playing trumpet
playing ukulele
playing violin
playing volleyball
playing xylophone
pole vault
presenting weather forecast
pull ups
pumping fist
pumping gas
punching bag
punching person (boxing)
push up
pushing car
pushing cart
pushing wheelchair
reading book
reading newspaper
recording music
riding a bike
riding camel
riding elephant
riding mechanical bull
riding mountain bike
riding mule
riding or walking with horse
riding scooter
riding unicycle
ripping paper
robot dancing
rock climbing
rock scissors paper
roller skating
running on treadmill
sailing
salsa dancing
sanding floor
scrambling eggs
scuba diving
setting table
shaking hands
shaking head
sharpening knives
sharpening pencil
shaving head
shaving legs
shearing sheep
shining shoes
shooting basketball
shooting goal (soccer)
shot put
shoveling snow
shredding paper
shuffling cards
side kick
sign language interpreting
singing
situp
skateboarding
ski jumping
skiing (not slalom or crosscountry)
skiing crosscountry
skiing slalom
skipping rope
skydiving
slacklining
slapping
sled dog racing
smoking
smoking hookah
snatch weight lifting
sneezing
sniffing
snorkeling
snowboarding
snowkiting
snowmobiling
somersaulting
spinning poi
spray painting
spraying
springboard diving
squat
sticking tongue out
stomping grapes
stretching arm
stretching leg
strumming guitar
surfing crowd
surfing water
sweeping floor
swimming backstroke
swimming breast stroke
swimming butterfly stroke
swing dancing
swinging legs
swinging on something
sword fighting
tai chi
taking a shower
tango dancing
tap dancing
tapping guitar
tapping pen
tasting beer
tasting food
testifying
texting
throwing axe
throwing ball
throwing discus
tickling
tobogganing
tossing coin
tossing salad
training dog
trapezing
trimming or shaving beard
trimming trees
triple jump
tying bow tie
tying knot (not on a tie)
tying tie
unboxing
unloading truck
using computer
using remote controller (not gaming)
using segway
vault
waiting in line
walking the dog
washing dishes
washing feet
washing hair
washing hands
water skiing
water sliding
watering plants
waxing back
waxing chest
waxing eyebrows
waxing legs
weaving basket
welding
whistling
windsurfing
wrapping present
wrestling
writing
yawning
yoga
zumba""".split('\n')

In [79]:
# classify_video(video_dir='/Users/aziai/Downloads/vtest/tmp/',
#                video_name='hi',
#                class_names=[],
#                model=,
#                opt=Opt())

In [26]:
from data import Video, make_dataset

In [81]:
spatial_transform = Compose([Scale(opt.sample_size),
                                 CenterCrop(opt.sample_size),
                                 ToTensor(),
                                 Normalize(opt.mean, [1, 1, 1])])

In [82]:
v = Video('/Users/aziai/Downloads/vtest/tmp',
         spatial_transform=spatial_transform)

In [83]:
opt.sample_size

112

In [84]:
v.__getitem__(0)[0].shape

torch.Size([3, 16, 112, 112])

### kd